# Last time: 
* review of mutable vs immutable, plus some more funny stuff
* selection sorting

# Today
* making our selection sort code better
* algorithm complexity

# Selection Sort revisited

We want to write a function that will sort a list of numbers:

e.g. we want `sort([2,15,-1,8,7])` 
to return `[15,8,7,2,-1]`

Idea for algorithm:
Move the maximum element to the front of the list, then move the maximum of the rest to the front of the rest, ...

Pseudo-code first level:
```
N = length of input list, xs
for i=0,...,N-1
    mloc = the location of the maximum of xs from i to N-1
    swap xs[mloc] and xs[i]
```

Let us begin by coding up `location of the maximum of xs from i to N-1`:
    

In [5]:
def argmax(xs, start, end):  
    '''returns the index of the max of a sub-list'''

    current_max = xs[start]
    current_max_location = start
    for i in range(start, end):
        if current_max < xs[i]:
            current_max = xs[i]
            current_max_location = i
    return current_max_location 

In [8]:
argmax([1,2,3,4,5],0,5)

4

In [6]:
argmax([6,2,3,4,5],0,2)

0

In [7]:
argmax([6,2,3,4,5],2,3)

2

During the sorting, we'll also need to swap things. Let's make that into a function too:

In [9]:
# note that this swaps *in place*
def swap(xs, i, j):
    dum = xs[i]
    xs[i] = xs[j]
    xs[j] = dum
    
# let's test:
xs = [1,2,3]
swap(xs,0,1)
print xs

[2, 1, 3]


Finally, we are ready to implement selection sort:

In [11]:
def sort(xs):
    N = len(xs)
    for i in range(N):
        swap(xs, argmax(xs,i,N), i)
    return xs

sort([2,15,-1,8,7])

[15, 8, 7, 2, -1]

Note that we could have written the same algorithm in one go like this:

In [13]:
def sort_difficult_to_read_and_debug(xs):
    N = len(xs)
    for i in range(N):
        current_max = xs[i]
        current_max_location = i
        for j in range(i, N):
            if current_max < xs[j]:
                current_max = xs[j]
                current_max_location = j
        dum = xs[i]
        xs[i] = xs[current_max_location]
        xs[current_max_location] = dum
    return xs

sort_difficult_to_read_and_debug([2, 15 ,-1 ,8 ,7])

[15, 8, 7, 2, -1]

But it is harder to read, and more importantly, there are no parts (functions) that you can test independently to make sure they are ok. So it's better to break the problem down to smaller parts (functions). 

### Improvements to our implementation

* We have implemented `argmax` such that it can operate on a sub-list of its input list. An alternative, that is slightly more readable, is to pass the sub-list directly to `argmax` and have it operate on the whole sub-list. The trade-off is that we then need to correct the index that `argmax` returns. 

* Again in `argmax`, when we cycle through the list, we keep track only of the index in the list, which we then use to find the corresponding element of the list. We can eliminate this extra step of indexing into the list via the syntax `for i,x in enumerate(xs):`, which tracks *both* the elements and their indices. 



In [19]:
def argmax(xs):  
    # notice the function signature has been simplified
    # the signature of the function is now 'argmax(xs)'
    # previously it was 'argmax(xs, start, end)'
    '''returns the index of the max of a sub-list'''

    current_max = xs[0]
    current_max_location = 0
    # use enumerate to cycle through i and xs[i] at the same time
    for i, x in enumerate(xs):   
        if current_max < x:
            current_max = x
            current_max_location = i
    return current_max_location 


In `sort`, we need to pass the sublist of `xs` that starts at index `i`. How do we do this? 

Python **slicing** to the rescue! Recall that the syntax `xs[i:j]` fetches the sublist of `xs` from the index on the left to the index on the right. 

Here are some examples to remind you:



In [41]:
xs = [1,2,3,4,5,6,7]
print "the element at index 2 is:  ", xs[2]
print "the sublist from indices 2 (included) to 4 (not included) is:  ", xs[2:4]
print "the part up to index 2 (not included) is:  ", xs[:2]
print "the part starting from index 2 (included) is:  ", xs[2:]


the element at index 2 is:   3
the sublist from indices 2 (included) to 4 (not included) is:   [3, 4]
the part up to index 2 (not included) is:   [1, 2]
the part starting from index 2 (included) is:   [3, 4, 5, 6, 7]


Let's set up a list and a sublist of it:

In [32]:
xs = [1,4,6,4,1,5,10]
xs_sublist = xs[2:5]
xs_sublist

[6, 4, 1]

We can find the location of an element in the original list using its location in the sublist and the location of the sublist in the original list: 

In [31]:
# fetch the element 6: 
print xs_sublist[0]
print xs[2+0]

6
6


Now, we are ready to re-implement `sort`:

In [38]:
def sort(xs):
    for i in range(len(xs)):
        # notice how we correct the index argmax returns:
        swap(xs, i + argmax(xs[i:]), i)
    return xs

sort([2,15,-1,8,7])

[15, 8, 7, 2, -1]

**Remark:** We didn't deal with empty lists in our code. We probably should. 

# Complexity

If I run `sort(xs)` on a list `xs` of length $n$. How many operations are done by Python? 

If we look through the code, we can see that we loop through the list once, and for each time we loop through, we loop through the part from `xs[i]` to `xs[n-1]`, where `n` is the list length. And then we do a few operations for swapping etc. All in all, it is safe to say that we do less than 
$$20\frac{n(n+1)}{2} = 10n^2 + 10n$$
operations. 

As $n$ becomes large, $n^2$ is clearly much much bigger than $n$, and the factor of $10$ is not that interesting, since it's not the real source of growth in the amount of time it will take. 

So we say the algorithm takes time of $\operatorname{O}(n^2)$. 

Officially, let us denote by $T(n)$ the longest number of steps that the algorithm takes on a list of length $n$, so $T: \mathbb{N} \rightarrow \mathbb{N}$ is function called the worst-case-time-complexity of the algorithm.
$T$ is $\operatorname{O}(n^k)$ if there is a constant $C$ such that, for large enough $n$, we have:
$$T(n) < Cn^k$$

Conclusion: the running time of the selection sort algorithm is (for large $n$) less than $Cn^2$, so we say it's an $\operatorname{O}(n^2)$ algorithm. 

By the way, the best sorting algorithm is $\operatorname{O}(n\log n)$.

# Questions and exercises

* What is the worst-case complexity of finding the maximum of a list?
* [Insertion sort](https://en.wikipedia.org/wiki/Insertion_sort). Code up the following sorting algorithm: take a list `xs`, make a new list `ys = []`. For each element `x` in `xs`, insert `x` into `ys` in a way so that `ys` stays sorted. e.g. if `ys = [10,8,4,1]` and we are inserting `x = 5`, `ys` will be `[10,8,5,4,1]`. (you can use `ys.insert(place, new_element)` if you want or write it yourself) 
* What is the worst-case complexity of insertion-sort?